In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,BertModel
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

c:\Users\visva\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

dataset = pd.read_csv('dreams.csv')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertmodel = BertModel.from_pretrained('bert-base-uncased')

def encode(text):
    input = tokenizer(text,return_tensors='pt',truncation=True,padding=True,max_length=512)
    with torch.no_grad():
        output = bertmodel(**input)
        hidden_state = output.last_hidden_state
        cls_embeddings = hidden_state[0,0,:]
    return cls_embeddings.numpy()

x = np.vstack([encode(t)for t in dataset['text']])
y = dataset['label'].values

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

x_train,x_test,y_train,y_test = train_test_split(x,y_encoded,test_size=0.2)

model = Sequential([
    Input(shape=(768,)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20,verbose=1)

text = 'i was running away from a guy with a knife '

encoded_text = encode(text).reshape(1,-1)
prediction = model.predict(encoded_text)
index = np.argmax(prediction)
confidence = prediction[0][index]

predicted_label = encoder.inverse_transform([int(index)])[0]
print(f"Predicted Sentiment: {predicted_label} \nConfidence: {confidence:.2f}")

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0703 - loss: 2.8229  
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1221 - loss: 2.5013 
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3792 - loss: 2.2745 
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5151 - loss: 2.0052 
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5893 - loss: 1.7685 
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7226 - loss: 1.5420 
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7404 - loss: 1.3590 
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8030 - loss: 1.1181 
Epoch 9/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 0.9840 
Epoch 10/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8882 - loss: 0.8197 
Epoch 11/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8722 - loss: 0.7296 
Epoch 12/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9056 - loss: 0.6204 
